# Dataframe Collection 

In [ ]:
import requests
import json
import os
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import pandas as pd

In [ ]:
f = open('sf_creds_prod.json')
creds = json.load(f)

In [ ]:
url = "https://login.salesforce.com/services/oauth2/token" #login
response = requests.post(f"{url}", data=creds)

In [ ]:
access_token = response.json().get("access_token")
instance_url = response.json().get("instance_url")

In [ ]:
response = requests.get(f"{instance_url}{os.environ.get("ENDPOINT")}",headers={'Content-type':'application/json', "Authorization":"Bearer " + access_token}, params={'limit': 2000})

In [ ]:
data_dict = {}

In [ ]:
for i in response.json()['columns']:
    data_dict[i['fieldNameOrPath']] = []

In [ ]:
for record in response.json()['records']:
    for col in record['columns']:
        key_value = list(col.values())
        data_dict[key_value[0]].append(key_value[1])


In [ ]:
df = pd.DataFrame.from_dict(data_dict)

In [ ]:
df['Award_Date__c'] = None
df['Awarded Budget'] = 0
df['Cost Ext?'] = False
df['Start_Date__c'] = None
df['End_Date__c'] = None
df['Cost Ext #'] = 0

In [ ]:
# df.at[0 ,'Cost Ext?'] = True #Set value in base DF
# row_copy = df.iloc[0] #Make a copy on index
# row_copy['Cost Ext?'] = True #Set Val on Copy
# df.loc[len(df)] = row_copy #Append copy to main

In [ ]:
new_rows = []
for index, row in df.iterrows():
    response_budget = requests.get(f"{instance_url}/services/data/v59.0/sobjects/Future_Gift__c/{row['Id']}/Life_of_Awards__r",headers={'Content-type':'application/json', "Authorization":"Bearer " + access_token})
    data = response_budget.json()
    if data['totalSize'] > 0:
        cost_mod_count = 1
        for budget in data['records']:
            if budget['Base_Award_Cost_Mod__c'] == 'Base Award':
                df.at[index , 'Awarded Budget'] += budget['Budget__c']
                df.at[index , 'Start_Date__c'] = budget['Start_Date__c']
                df.at[index , 'End_Date__c'] = budget['End_Date__c']
                df.at[index , 'Award_Date__c'] = budget['Award_Date__c']
            elif budget['Base_Award_Cost_Mod__c'] == 'Cost Modification':
                row_copy = df.iloc[index]
                row_copy['Cost Ext?'] = True
                row_copy['Cost Ext #'] = cost_mod_count
                row_copy['Awarded Budget'] = budget['Budget__c']
                row_copy['Start_Date__c'] = budget['Start_Date__c']
                row_copy['End_Date__c'] = budget['End_Date__c']
                row_copy['Award_Date__c'] = budget['Award_Date__c']
                cost_mod_count += 1
                new_rows.append(row_copy)

In [ ]:
for i in new_rows:
    df.loc[len(df)] = i

In [ ]:
df['Award_Date__c'] = pd.to_datetime(df['Award_Date__c'])
df['Status_Date__c'] = pd.to_datetime(df['Status_Date__c'], utc=True)
df['Submission_Didn_t_Submit_Date__c'] = pd.to_datetime(df['Submission_Didn_t_Submit_Date__c'], utc=True)

df['Amount__c'] = pd.to_numeric(df['Amount__c'])
df['Total_Donor_Budget_Submitted__c'] = pd.to_numeric(df['Total_Donor_Budget_Submitted__c'])

In [ ]:
df['CreatedDate'] = pd.to_datetime(df['CreatedDate'], utc=True)

In [ ]:
import datetime

In [ ]:
currentDay = datetime.datetime.now().day
currentMonth = datetime.datetime.now().month
currentYear = datetime.datetime.now().year

current_day = str(datetime.date(currentYear, currentMonth, currentDay))

if currentMonth == 1:
    analyzedMonth = 12
    analyzedMonthsYear = currentYear - 1
else:
    analyzedMonth = currentMonth - 1
    analyzedMonthsYear = currentYear

if analyzedMonth == 1:
    prevMonth = 12
    prevMonthsYear = analyzedMonthsYear - 1
else:
    prevMonth = analyzedMonth - 1
    prevMonthsYear = analyzedMonthsYear

In [ ]:
def last_day_of_month(any_day):
    next_month = any_day.replace(day=28) + datetime.timedelta(days=4)
    return next_month - datetime.timedelta(days=next_month.day)

In [ ]:
current_month_start = str(datetime.date(currentYear, currentMonth, 1))
current_month_start_previous_year = str(datetime.date(currentYear - 1, currentMonth, 1))
current_month_end = str(last_day_of_month(datetime.date(currentYear, currentMonth, 1)))
current_month_start_analyzed_fy = str(datetime.date((currentYear - 1), currentMonth, 1))
if currentMonth < 10:
    current_FY_start = str(datetime.date((currentYear - 1), 10, 1))
    previous_FY_start = str(datetime.date((currentYear - 2), 10, 1))
else:
    current_FY_start = str(datetime.date(currentYear, 10, 1))
    previous_FY_start = str(datetime.date(currentYear - 1, 10, 1))

analyzed_month_start = str(datetime.date(analyzedMonthsYear, analyzedMonth, 1))
analyzed_month_end = str(last_day_of_month(datetime.date(analyzedMonthsYear, analyzedMonth, 1)))
analyzed_month_end_previous_year = str(last_day_of_month(datetime.date(analyzedMonthsYear - 1, analyzedMonth, 1)))

previous_month_start = str(datetime.date(prevMonthsYear, prevMonth, 1))
previous_month_end = str(last_day_of_month(datetime.date(prevMonthsYear, prevMonth, 1)))

In [ ]:
print(
    [f'The current day is {current_day}\n\n'],
    [f'The current month is {current_month_start} to {current_month_end}\n'],
    [f'The analyzed month is {analyzed_month_start} to {analyzed_month_end}\n'],
    [f'The previous month is {previous_month_start} to {previous_month_end}\n\n'],
    [f'Our current FY start date is {current_FY_start} and analyzed FYTD is {current_FY_start} to {analyzed_month_end}\n'],
    [f'Our last FY start date is {previous_FY_start} and previous analyzed FYTD is {previous_FY_start} to {analyzed_month_end_previous_year}\n\n'],
    [f'The 12 month timeframe is from {current_month_start_previous_year} to {current_month_start}']
    )

# METRICS

In [ ]:
class SalesforceData:
    def __init__(self, start, end, current_day, df):
        self.start_date = start
        self.end_date = end
        self.submitted = df[(df['Submission_Didn_t_Submit_Date__c'] >= start) & (df['Submission_Didn_t_Submit_Date__c'] <= current_day)]
        #self.awarded_data = df[(df['Award_Date__c'] >= start) & (df['Award_Date__c'] <= end)]
        self.awarded_data = df[(df['Start_Date__c'] >= start) & (df['Start_Date__c'] <= end)]
        self.not_awarded_data = df[(df['Status__c'] == "Not Awarded") & (df['Status_Date__c'] >= start) & (df['Status_Date__c'] <= end)]
        self.pipeline = df[(df['Status__c'].isin(['Business Development', 'Under Donor Review', 'Proposal Development']))]
        self.cos =['Bangladesh', 'Bolivia', 'Burundi', 'Cambodia',
       'Democratic Republic of the Congo', 'Dominican Republic',
       'Ethiopia', 'Guatemala', 'Haiti', 'Indonesia', 'Kenya',
       'Mozambique', 'Nicaragua', 'Peru',
       'Philippines', 'Rwanda', 'South Sudan', 'Uganda', 'United States', None]
    
    def dataset(self, type, cost_ext=None, document=None, funding_type=None, country=None, sector=None, min_max=None, status=None, awarded=True, calculate=True):
        
        def valid_error(item, valid_list):
            if item not in valid_list:
                raise ValueError(item + " must be in %r." % valid_list)
        
        valid_error(type, [str, int, "count"])
        valid_error(cost_ext, [True, False, None])
        valid_error(funding_type, ['Private Contractual', 'Multilateral', 'USG',
       'Other Private', None])
        valid_error(country, ['Bahamas', 'Bangladesh', 'Bolivia', 'Burundi', 'Cambodia',
       'Democratic Republic of the Congo', 'Dominican Republic', 'Egypt',
       'Ethiopia', 'Guatemala', 'Haiti', 'India', 'Indonesia', 'Kenya',
       'Lebanon', 'Mozambique', 'Nicaragua', 'Niger', 'Palestine', 'Peru',
       'Philippines', 'Rwanda', 'South Sudan', 'Uganda', 'United States', 'Other', None])
        valid_error(sector, ['Other', 'Education','WASH','Food Security', 'Livelihoods','Health', 'Nutrition', 
        'Leadership, Democracy & Governance', 'Gender Equity & Social Inclusion','Climate & Environment',
        'DRR','Child Protection','Infrastructure','Capacity Development','Social and Behavioral Change', None])
        valid_error(awarded, [True, False, "pipeline", "submitted"])
        valid_error(calculate, [True, False, ])
        
        if awarded == 'pipeline':
            if status not in ['Business Development', 'Under Donor Review', 'Proposal Development', None]:
                raise ValueError("With awarded='pipeline', " + status + " must be in %r." % ['Business Development', 'Under Donor Review', 'Proposal Development'])
            
            filtered_data = self.pipeline[self.pipeline['Full_Proposal__c'] == "Yes"] if (document == "FP") else self.pipeline[self.pipeline['Concept_Note__c'] == "Yes"] if (document == "CN") else self.pipeline
            filtered_data = filtered_data[filtered_data['Status__c'] == status] if (status != None) else filtered_data
            budget_key = 'Total_Donor_Budget_Submitted__c' if status == "Under Donor Review" else "Amount__c"

        elif awarded == 'submitted':
            filtered_data = self.submitted[self.submitted['Full_Proposal__c'] == "Yes"] if (document == "FP") else self.submitted[self.submitted['Concept_Note__c'] == "Yes"] if (document == "CN") else self.submitted
            budget_key = 'Total_Donor_Budget_Submitted__c'
        elif awarded == True:
            filtered_data = self.awarded_data[self.awarded_data['Cost Ext?'] == cost_ext] if (cost_ext != None) else self.awarded_data
            budget_key = 'Awarded Budget'

        elif awarded == False:
            filtered_data = self.not_awarded_data[self.not_awarded_data['Full_Proposal__c'] == "Yes"] if (document == "FP") else self.not_awarded_data[self.not_awarded_data['Concept_Note__c'] == "Yes"] if (document == "CN") else self.not_awarded_data
            budget_key = 'Total_Donor_Budget_Submitted__c'

        filtered_data = filtered_data[filtered_data['Funding_Type__c'] == funding_type] if (funding_type != None) else filtered_data
        filtered_data = filtered_data[~filtered_data['Geographic_Location__r.Name'].isin(self.cos)] if (country == "Other") else filtered_data[filtered_data['Geographic_Location__r.Name'] == country] if (country != None) else filtered_data
        filtered_data = filtered_data[filtered_data['Primary_Sector__c'] == sector] if (sector != None) else filtered_data
        filtered_data =  filtered_data[(filtered_data[budget_key] > min_max[0]) & (filtered_data[budget_key] < min_max[1])] if (min_max != None) else filtered_data

        if calculate:
            filtered_data = filtered_data[budget_key]
            filtered_data ='${:,.2f} '.format(filtered_data.sum()/1000000) if (type == str) else len(filtered_data) if type == "count" else filtered_data.sum()/1000

        return filtered_data
    
    def win_rate(self, type, calculate=True, **kwargs):
        
        valid_type = ["#", "$"]
        if type not in valid_type:
            raise ValueError("Type must be one of %r." % valid_type)
        type = int if (type == "$") else "count"

        if calculate:
            awarded_sum = SalesforceData.dataset(self, type=type, **kwargs)
            try:
                calc = awarded_sum / (awarded_sum + SalesforceData.dataset(self, type=type, document="FP", awarded=False, **kwargs))
                result = "" if pd.isna(calc) else "{:.2%}".format(calc)
            except:
                result = ""
        else:
            df_awarded = SalesforceData.dataset(self, type=type, calculate=False, **kwargs)
            df_not_awarded = SalesforceData.dataset(self, type=type, document="FP", awarded=False, calculate=False, **kwargs)
            summary_list = []
            for df in [df_awarded, df_not_awarded]:
                names = df['Name']
                summary_list.append("\n".join(names.tolist()))
            result = {
                "awarded": df_awarded ,
                "not_awarded": df_not_awarded,
                "summary": lambda: print(f'Considering the provided arguments, the following opportunities were awarded:\n\n{summary_list[0]}\n\nWhile the following opportunities were not awarded\n\n{summary_list[1]}')
            }
        return result

    def perc_usg(self, type, calculate=True, **kwargs):

        valid_type = ["#", "$"]
        if type not in valid_type:
            raise ValueError("Type must be one of %r." % valid_type)
        type = int if (type == "$") else "count"

        if calculate:
            calc = SalesforceData.dataset(self, type=type, funding_type="USG", **kwargs)/SalesforceData.dataset(self, type=type, **kwargs)
            result = "0%" if pd.isna(calc) else calc
        else:
            df_usg = SalesforceData.dataset(self, type=type, funding_type="USG", calculate=False, **kwargs)
            df_all = SalesforceData.dataset(self, type=type, calculate=False, **kwargs)
            summary_list = []
            for df in [df_usg, df_all]:
                names = df['Name']
                summary_list.append("\n".join(names.tolist()))
            result = {
                "usg": df_usg ,
                "non_usg": df_all,
                "summary": lambda: print(f'Considering the provided arguments, the following opportunities are USG:\n\n{summary_list[0]}\n\nOut of the following opportunities\n\n{summary_list[1]}')
            }

        return result

In [ ]:
last_ytd = SalesforceData(previous_FY_start, analyzed_month_end_previous_year, current_day, df)
ytd = SalesforceData(current_FY_start, analyzed_month_end, current_day, df)
past_12 = SalesforceData(analyzed_month_end_previous_year, analyzed_month_end, current_day, df)
month= SalesforceData(analyzed_month_start, analyzed_month_end, current_day, df)
last_month = SalesforceData(previous_month_start, previous_month_end, current_day, df)

## Universe of USG Funding

In [ ]:
data = pd.read_csv("https://www.usaid.gov/business-forecast/export/csv?page&_format=csv")
df_usaid = pd.DataFrame(data)

new_df_usaid = df_usaid[(df_usaid["Operating Unit"].isin([
        'Bangladesh',
        'Bolivia',
        'Burundi',
        'Cambodia',
        'Democratic Republic of the Congo',
        'Dominican Republic',
        'Ethiopia',
        'Guatemala',
        'Haiti',
        'Indonesia',
        'Kenya',
        'Mozambique',
        'Nicaragua',
        'Peru',
        'Philippines',
        'Rwanda',
        'South Sudan',
        'Uganda',
        'Bureau for Conflict Prevention and Stabilization (CPS)',
        "Regional Development Mission for Asia",
        "Bureau for Humanitarian Assistance (BHA)",
        "Innovation, Technology, and Research Hub (DDI/ITR)",
        "Private-Sector Engagement Hub (DDI/PSE)",
        "Bureau for Resilience and Food Security (RFS)",
        "Local, Faith, and Transformative Partnerships Hub (DDI/LFT)",
        "Center for Environment, Energy, and Infrastructure (DDI/EEI)",
        "Center for Democracy, Human Rights, and Governance (DDI/DRG)",
        "Office of Population & Reproductive Health (GH/PRH)",
        "Office of Maternal and Child Health and Nutrition (GH/MCHN)",
        "Gender Equality and Women's Empowerment Hub (DDI/GEWE)",
        "Center for Accelerating Innovation and Impact (GH/CAII)",
        "Inclusive Development Hub (DDI/ID)",
        "Office of Acquisition and Assistance (M/OAA)"
])) 
# & (~df_usaid['Award/Action Type'].isin(['Grant',
#        'Cooperative Agreement', 'To Be Determined (TBD)']))
]


new_ranges = []
for i in new_df_usaid["Total Estimated Cost/Amount Range"]:
    if ".99M" in i:
        i = i.replace(".99M",",000,000")
    if ".49B" in i:
        i = i.replace(".49B",",000,000,000")
    if ".79B" in i:
        i = i.replace(".79B",",000,000,000")
    if ".5B" in i:
        i = i.replace(".5B",",000,000,000")
    if "B+" in i:
        i = i.replace("B+",",000,000,000")
    if "B" in i:
        i = i.replace("B",",000,000,000")
    if "M" in i:
        i = i.replace("M",",000,000")
    new_ranges.append(i.replace("$","").replace(",",""))

new_df_usaid["New Ranges"] = new_ranges


min = []
max = []
for i in new_df_usaid["New Ranges"]:
    if "-" in i:
        arr = i.split(" - ")
        min.append(int(arr[0]))
        max.append(int(arr[1]))
    else:
        min.append(0)
        max.append(int(i))

new_df_usaid["Min"] = min
new_df_usaid["Max"] = max

universe_of_funding = new_df_usaid['Min'].sum()

## Google Sheets

In [ ]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = os.environ.get("SPREADSHEETID")

creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "creds.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

def batch_update_values(data):
  try:
    service = build("sheets", "v4", credentials=creds)

    body = {"valueInputOption": "USER_ENTERED", "data": data}
    result = (
        service.spreadsheets()
        .values()
        .batchUpdate(spreadsheetId=SAMPLE_SPREADSHEET_ID, body=body)
        .execute()
    )
    print(f"{(result.get('totalUpdatedCells'))} cells updated.")
    return result
  except HttpError as error:
    print(f"An error occurred: {error}")
    return error

In [ ]:
africa_list = ['Burundi','Democratic Republic of the Congo','Ethiopia','Kenya','Mozambique','Rwanda','South Sudan','Uganda']
asia_list = ['Bangladesh','Cambodia','Indonesia','Philippines']
lac_list = ['Bolivia','Dominican Republic','Guatemala','Haiti','Nicaragua','Peru']
sectors = ['Child Protection','DRR','Education','Food Security', 'Livelihoods', 'Health','Leadership, Democracy & Governance','WASH','Other','Other']
sectors_2 = ["Capacity Development","Child Protection","Climate & Environment","DRR","Education","Food Security","Gender Equity & Social Inclusion","Health","Leadership, Democracy & Governance","Livelihoods","Nutrition","WASH"]
funding_types = ["USG", "Multilateral", "Private Contractual", "Other Private"]
pipelines = ['Business Development', 'Proposal Development', 'Under Donor Review']
ranges = [[0, 250_000], [250_000, 500_000], [500_000, 1_000_000], [1_000_000, 5_000_000], [5_000_000, 25_000_000], [25_000_000, 1_000_000_000_000_000]]

def expand_list(list, filter_in_list, obj, **kwargs):
    values = []
    for i in list:
        kwargs[filter_in_list] = i
        values.append([round(obj(**kwargs)) if type(obj(**kwargs)) != str else obj(**kwargs)])
    return values

In [ ]:
data = [
        {"range": "Date Filters!A1:A7", 
         "values": [[f'The current day is {current_day}\n\n'],
                    [f'The current month is {current_month_start} to {current_month_end}\n'],
                    [f'The analyzed month is {analyzed_month_start} to {analyzed_month_end}\n'],
                    [f'The previous month is {previous_month_start} to {previous_month_end}\n\n'],
                    [f'Our current FY start date is {current_FY_start} and analyzed FYTD is {current_FY_start} to {analyzed_month_end}\n'],
                    [f'Our last FY start date is {previous_FY_start} and previous analyzed FYTD is {previous_FY_start} to {analyzed_month_end_previous_year}\n\n'],
                    [f'The 12 month timeframe is from {current_month_start_previous_year} to {current_month_start}']
                    ]
         },
        {"range": "1. FY24 Overview!C2:C5", 
         "values": [[f'{month.start_date} to {month.end_date}'],
                    [month.dataset(str, cost_ext=False) + f'({month.dataset("count", cost_ext=False)})'], 
                    [month.dataset(str, cost_ext=True) + f'({month.dataset("count", cost_ext=True)})'], 
                    [month.dataset(str) + f'({month.dataset("count")})']]
         },
        {"range": "1. FY24 Overview!D2:D5", 
         "values": [[f'{last_month.start_date} to {last_month.end_date}'],
                    [last_month.dataset(str, cost_ext=False) + f'({last_month.dataset("count", cost_ext=False)})'], 
                    [last_month.dataset(str, cost_ext=True) + f'({last_month.dataset("count", cost_ext=True)})'], 
                    [last_month.dataset(str) + f'({last_month.dataset("count")})']]
         },
        {"range": "1. FY24 Overview!E2:E5", 
         "values": [[f'{ytd.start_date} to {ytd.end_date}'],
                    [ytd.dataset(str, cost_ext=False) + f'({ytd.dataset("count", cost_ext=False)})'], 
                    [ytd.dataset(str, cost_ext=True) + f'({ytd.dataset("count", cost_ext=True)})'], 
                    [ytd.dataset(str) + f'({ytd.dataset("count")})']]
        },
        {"range": "1. FY24 Overview!F2:F5", 
         "values": [[f'{last_ytd.start_date} to {last_ytd.end_date}'],
                    [last_ytd.dataset(str, cost_ext=False) + f'({last_ytd.dataset("count", cost_ext=False)})'], 
                    [last_ytd.dataset(str, cost_ext=True) + f'({last_ytd.dataset("count", cost_ext=True)})'], 
                    [last_ytd.dataset(str) + f'({last_ytd.dataset("count")})']]
        },
        {"range": "1. FY24 Overview!C7:C9", 
         "values": [[month.win_rate("#")], 
                    [month.win_rate("$")], 
                    [month.win_rate("$", funding_type="USG")]]
        },
        {"range": "1. FY24 Overview!D7:D9", 
         "values": [[last_month.win_rate("#")], 
                    [last_month.win_rate("$")], 
                    [last_month.win_rate("$", funding_type="USG")]]
        },
        {"range": "1. FY24 Overview!E7:E9", 
         "values": [[ytd.win_rate("#")], 
                    [ytd.win_rate("$")], 
                    [ytd.win_rate("$", funding_type="USG")]]
        },
        {"range": "1. FY24 Overview!F7:F9", 
         "values": [[last_ytd.win_rate("#")], 
                    [last_ytd.win_rate("$")], 
                    [last_ytd.win_rate("$", funding_type="USG")]]
        },
        {"range": "1. FY24 Overview!C11:C12", 
         "values": [[month.perc_usg("$", cost_ext=False)], 
                    [month.perc_usg("$", cost_ext=True)]]
        },
        {"range": "1. FY24 Overview!D11:D12", 
         "values": [[last_month.perc_usg("$", cost_ext=False)], 
                    [last_month.perc_usg("$", cost_ext=True)]]
        },
        {"range": "1. FY24 Overview!E11:E12", 
         "values": [[ytd.perc_usg("$", cost_ext=False)], 
                    [ytd.perc_usg("$", cost_ext=True)]]
        },
        {"range": "1. FY24 Overview!F11:F12", 
         "values": [[last_ytd.perc_usg("$", cost_ext=False)], 
                    [last_ytd.perc_usg("$", cost_ext=True)]]
        },
        {"range": "2. CO New Business & Win %!D6:D13", 
         "values": expand_list(africa_list, "country", month.dataset, type=int)
        },
        {"range": "2. CO New Business & Win %!E6:E13", 
         "values": expand_list(africa_list, "country", ytd.dataset, type=int)
        },
        {"range": "2. CO New Business & Win %!F6:F13", 
         "values": expand_list(africa_list, "country", last_ytd.dataset, type=int)
        },
        {"range": "2. CO New Business & Win %!H6:H13", 
         "values": expand_list(africa_list, "country", month.win_rate, type="$")
        },
        {"range": "2. CO New Business & Win %!I6:I13", 
         "values": expand_list(africa_list, "country", ytd.win_rate, type="$")
        },
        {"range": "2. CO New Business & Win %!J6:J13", 
         "values": expand_list(africa_list, "country", last_ytd.win_rate, type="$")
        },
        {"range": "2. CO New Business & Win %!K6:K13", 
         "values": expand_list(africa_list, "country", past_12.win_rate, type="$")
        },
        {"range": "2. CO New Business & Win %!D15:D18", 
         "values": expand_list(asia_list, "country", month.dataset, type=int)
        },
        {"range": "2. CO New Business & Win %!E15:E18", 
         "values": expand_list(asia_list, "country", ytd.dataset, type=int)
        },
        {"range": "2. CO New Business & Win %!F15:F18", 
         "values": expand_list(asia_list, "country", last_ytd.dataset, type=int)
        },
        {"range": "2. CO New Business & Win %!H15:H18", 
         "values": expand_list(asia_list, "country", month.win_rate, type="$")
        },
        {"range": "2. CO New Business & Win %!I15:I18", 
         "values": expand_list(asia_list, "country", ytd.win_rate, type="$")
        },
        {"range": "2. CO New Business & Win %!J15:J18", 
         "values": expand_list(asia_list, "country", last_ytd.win_rate, type="$")
        },
        {"range": "2. CO New Business & Win %!K15:K18", 
         "values": expand_list(asia_list, "country", past_12.win_rate, type="$")
        },
        {"range": "2. CO New Business & Win %!D20:D25", 
         "values": expand_list(lac_list, "country", month.dataset, type=int)
        },
        {"range": "2. CO New Business & Win %!E20:E25", 
         "values": expand_list(lac_list, "country", ytd.dataset, type=int)
        },
        {"range": "2. CO New Business & Win %!F20:F25", 
         "values": expand_list(lac_list, "country", last_ytd.dataset, type=int)
        },
        {"range": "2. CO New Business & Win %!H20:H25", 
         "values": expand_list(lac_list, "country", month.win_rate, type="$")
        },
        {"range": "2. CO New Business & Win %!I20:I25", 
         "values": expand_list(lac_list, "country", ytd.win_rate, type="$")
        },
        {"range": "2. CO New Business & Win %!J20:J25", 
         "values": expand_list(lac_list, "country", last_ytd.win_rate, type="$")
        },
        {"range": "2. CO New Business & Win %!K20:K25", 
         "values": expand_list(lac_list, "country", past_12.win_rate, type="$")
        },
        {"range": "2. CO New Business & Win %!D27:K27", 
         "values": [[month.dataset(int, country="United States"),
                    ytd.dataset(int, country="United States"),
                    last_ytd.dataset(int, country="United States"),
                    "",
                    month.win_rate("$", country="United States"),
                    ytd.win_rate("$", country="United States"),
                    last_ytd.win_rate("$", country="United States"),
                    past_12.win_rate("$", country="United States")]]
        },
        {"range": "3. Sector New Business & Win %!C4:C13", 
         "values": expand_list(sectors, "sector", month.dataset, type=int)
        },
        {"range": "3. Sector New Business & Win %!D4:D13", 
         "values": expand_list(sectors, "sector", ytd.dataset, type=int)
        },
        {"range": "3. Sector New Business & Win %!E4:E13", 
         "values": expand_list(sectors, "sector", last_ytd.dataset, type=int)
        },
        {"range": "3. Sector New Business & Win %!F4:F13", 
         "values": expand_list(sectors, "sector", month.win_rate, type="$")
        },
        {"range": "3. Sector New Business & Win %!G4:G13", 
         "values": expand_list(sectors, "sector", ytd.win_rate, type="$")
        },
        {"range": "3. Sector New Business & Win %!H4:H13", 
         "values": expand_list(sectors, "sector", last_ytd.win_rate, type="$")
        },
        {"range": "3. Sector New Business & Win %!I4:I13", 
         "values": expand_list(sectors, "sector", past_12.win_rate, type="$")
        },
        {"range": "4. Donor New Business & Win %!B4:B7", 
         "values": expand_list(funding_types, "funding_type", month.dataset, type=int)
        },
        {"range": "4. Donor New Business & Win %!C4:C7", 
         "values": expand_list(funding_types, "funding_type", ytd.dataset, type=int)
        },
        {"range": "4. Donor New Business & Win %!D4:D7", 
         "values": expand_list(funding_types, "funding_type", last_ytd.dataset, type=int)
        },
        {"range": "4. Donor New Business & Win %!E4:E7", 
         "values": expand_list(funding_types, "funding_type", month.win_rate, type="$")
        },
        {"range": "4. Donor New Business & Win %!F4:F7", 
         "values": expand_list(funding_types, "funding_type", ytd.win_rate, type="$")
        },
        {"range": "4. Donor New Business & Win %!G4:G7", 
         "values": expand_list(funding_types, "funding_type", last_ytd.win_rate, type="$")
        },
        {"range": "4. Donor New Business & Win %!H4:H7", 
         "values": expand_list(funding_types, "funding_type", past_12.win_rate, type="$")
        },
        {"range": "5. Awards & Win % by Threshold!B4:B9", 
         "values": expand_list(ranges, "min_max", ytd.dataset, type="count")
        },
        {"range": "5. Awards & Win % by Threshold!C4:C9", 
         "values": expand_list(ranges, "min_max", ytd.dataset, type="count", awarded=False)
        },
        {"range": "5. Awards & Win % by Threshold!D4:D9", 
         "values": expand_list(ranges, "min_max", ytd.dataset, type=int)
        },
        {"range": "5. Awards & Win % by Threshold!E4:E9", 
         "values": expand_list(ranges, "min_max", ytd.dataset, type=int, awarded=False)
        },
        {"range": "5. Awards & Win % by Threshold!F4:F9", 
         "values": expand_list(ranges, "min_max", ytd.win_rate, type="#")
        },
        {"range": "5. Awards & Win % by Threshold!F4:F9", 
         "values": expand_list(ranges, "min_max", ytd.win_rate, type="$")
        },
        {"range": "6. QBR Pipeline Overview!C15", 
         "values": [[int(universe_of_funding)]]
        },
        {"range": "6. QBR Pipeline Overview!C4:C6", 
         "values": expand_list(pipelines, "status", ytd.dataset, type=int, awarded="pipeline")
        },
        {"range": "6. QBR Pipeline Overview!D4:D6", 
         "values": expand_list(pipelines, "status", ytd.dataset, type=int, awarded="pipeline", funding_type="USG")
        },
        {"range": "6. QBR Pipeline Overview!C8:C12",
         "values": [[ytd.dataset(int, awarded="submitted", document="CN")],
                    [ytd.dataset(int, awarded="submitted", document="FP")],
                    [ytd.dataset(int)],
                    [past_12.win_rate("$")],
                    [f"{past_12.dataset(str)}/{past_12.dataset(str, document='FP', awarded=False)}"]
                    ]
        },
        {"range": "6. QBR Pipeline Overview!D8:D12",
         "values": [[ytd.dataset(int, awarded="submitted", document="CN", funding_type="USG")],
                    [ytd.dataset(int, awarded="submitted", document="FP", funding_type="USG")],
                    [ytd.dataset(int, funding_type="USG")],
                    [past_12.win_rate("$", funding_type="USG")],
                    [f"{past_12.dataset(str, funding_type='USG')}/{past_12.dataset(str, document='FP', funding_type='USG', awarded=False)}"]
                    ]
        },
        {"range": "7. QBR Pipeline Geo Analysis!D5:D12", 
         "values": expand_list(africa_list, "country", ytd.dataset, type=int)
        },
        {"range": "7. QBR Pipeline Geo Analysis!D14:D17", 
         "values": expand_list(asia_list, "country", ytd.dataset, type=int)
        },
        {"range": "7. QBR Pipeline Geo Analysis!D19:D24", 
         "values": expand_list(lac_list, "country", ytd.dataset, type=int)
        },
        {"range": "7. QBR Pipeline Geo Analysis!D26:D27", 
         "values": [[ ytd.dataset(int, country="United States")],
                    [ ytd.dataset(int, country="Other")]]
        },
        {"range": "7. QBR Pipeline Geo Analysis!F5:F12", 
         "values": expand_list(africa_list, "country", ytd.dataset, type=int, awarded="submitted")
        },
        {"range": "7. QBR Pipeline Geo Analysis!F14:F17", 
         "values": expand_list(asia_list, "country", ytd.dataset, type=int, awarded="submitted")
        },
        {"range": "7. QBR Pipeline Geo Analysis!F19:F24", 
         "values": expand_list(lac_list, "country", ytd.dataset, type=int, awarded="submitted")
        },
        {"range": "7. QBR Pipeline Geo Analysis!F26:F27", 
         "values": [[ ytd.dataset(int, awarded="submitted", country="United States")],
                    [ ytd.dataset(int, awarded="submitted", country="Other")]]
        },
        {"range": "7. QBR Pipeline Geo Analysis!G5:G12", 
         "values": expand_list(africa_list, "country", ytd.dataset, type=int, status="Under Donor Review", awarded="pipeline")
        },
        {"range": "7. QBR Pipeline Geo Analysis!G14:G17", 
         "values": expand_list(asia_list, "country", ytd.dataset, type=int, status="Under Donor Review", awarded="pipeline")
        },
        {"range": "7. QBR Pipeline Geo Analysis!G19:G24", 
         "values": expand_list(lac_list, "country", ytd.dataset, type=int, status="Under Donor Review", awarded="pipeline")
        },
        {"range": "7. QBR Pipeline Geo Analysis!G26:G27", 
         "values": [[ ytd.dataset(int, awarded="pipeline", status="Under Donor Review", country="United States")],
                    [ ytd.dataset(int, awarded="pipeline", status="Under Donor Review", country="Other")]]
        },
        {"range": "7. QBR Pipeline Geo Analysis!H5:H12", 
         "values": expand_list(africa_list, "country", ytd.dataset, type=int, status="Proposal Development", awarded="pipeline")
        },
        {"range": "7. QBR Pipeline Geo Analysis!H14:H17", 
         "values": expand_list(asia_list, "country", ytd.dataset, type=int, status="Proposal Development", awarded="pipeline")
        },
        {"range": "7. QBR Pipeline Geo Analysis!H19:H24", 
         "values": expand_list(lac_list, "country", ytd.dataset, type=int, status="Proposal Development", awarded="pipeline")
        },
        {"range": "7. QBR Pipeline Geo Analysis!H26:H27", 
         "values": [[ ytd.dataset(int, awarded="pipeline", status="Proposal Development", country="United States")],
                    [ ytd.dataset(int, awarded="pipeline", status="Proposal Development", country="Other")]]
        },
        {"range": "7. QBR Pipeline Geo Analysis!I5:I12", 
         "values": expand_list(africa_list, "country", ytd.dataset, type=int, status="Business Development", awarded="pipeline")
        },
        {"range": "7. QBR Pipeline Geo Analysis!I14:I17", 
         "values": expand_list(asia_list, "country", ytd.dataset, type=int, status="Business Development", awarded="pipeline")
        },
        {"range": "7. QBR Pipeline Geo Analysis!I19:I24", 
         "values": expand_list(lac_list, "country", ytd.dataset, type=int, status="Business Development", awarded="pipeline")
        },
        {"range": "7. QBR Pipeline Geo Analysis!I26:I27", 
         "values": [[ ytd.dataset(int, awarded="pipeline", status="Business Development", country="United States")],
                    [ ytd.dataset(int, awarded="pipeline", status="Business Development", country="Other")]]
        },
        {"range": "8. QBR Pipeline Sector Analysis!B3:B14", 
         "values": expand_list(sectors_2, "sector", ytd.dataset, type=int)
        },
        {"range": "8. QBR Pipeline Sector Analysis!C3:C14", 
         "values": expand_list(sectors_2, "sector", ytd.dataset, type=int, status="Under Donor Review", awarded="pipeline")
        },
        {"range": "8. QBR Pipeline Sector Analysis!D3:D14", 
         "values": expand_list(sectors_2, "sector", ytd.dataset, type=int, status="Proposal Development", awarded="pipeline")
        },
        {"range": "8. QBR Pipeline Sector Analysis!E3:E14", 
         "values": expand_list(sectors_2, "sector", ytd.dataset, type=int, status="Business Development", awarded="pipeline")
        },
        {"range": "8. QBR Pipeline Sector Analysis!B24:B35", 
         "values": expand_list(sectors_2, "sector", ytd.dataset, type="count")
        },
        {"range": "8. QBR Pipeline Sector Analysis!C24:C35", 
         "values": expand_list(sectors_2, "sector", ytd.dataset, type="count", status="Under Donor Review", awarded="pipeline")
        },
        {"range": "8. QBR Pipeline Sector Analysis!D24:D35", 
         "values": expand_list(sectors_2, "sector", ytd.dataset, type="count", status="Proposal Development", awarded="pipeline")
        },
        {"range": "8. QBR Pipeline Sector Analysis!E24:E35", 
         "values": expand_list(sectors_2, "sector", ytd.dataset, type="count", status="Business Development", awarded="pipeline")
        },
        {"range": "9. Submissions!C4:C15", 
         "values": expand_list(sectors_2, "sector", ytd.dataset, type=int, awarded="submitted")
        },
        {"range": "9. Submissions!D4:D15", 
         "values": expand_list(sectors_2, "sector", last_ytd.dataset, type=int, awarded="submitted")
        },
        {"range": "9. Submissions!C21:C24", 
         "values": expand_list(funding_types, "funding_type", ytd.dataset, type=int, awarded="submitted")
        },
        {"range": "9. Submissions!D21:D24", 
         "values": expand_list(funding_types, "funding_type", last_ytd.dataset, type=int, awarded="submitted")
        },
        {"range": "9. Submissions!C33:C39", 
         "values": expand_list(["Capacity Development","Education","Food Security","Health","Leadership, Democracy & Governance","Livelihoods","WASH"], "sector", ytd.dataset, type=int, awarded="submitted")
        },
        {"range": "9. Submissions!D33:D39", 
         "values": expand_list(["Capacity Development","Education","Food Security","Health","Leadership, Democracy & Governance","Livelihoods","WASH"], "sector", last_ytd.dataset, type=int, awarded="submitted")
        }
    ]

In [ ]:
batch_update_values(data)